In [3]:
# importing the required library
from google.colab import drive

# mounting google drive to Collab
drive.mount("/content/drive")
# moving around in google drive
%cd drive/MyDrive
# Listing all the files/folders present
%ls
# moving around in google drive
%cd  Contents/
# Listing all the files/folders present
%ls

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# installing the pyspark library.
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e0c3af17deda50b34922e9066e3a8c05e46268f4e529309d2223257fcbf80a41
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# importing SparkSession from pyspark.sql library
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, regexp_extract, regexp_replace, split

# What is the differnece between SparkSession and SparkContext
# SparkContext was the entry point for programming Spark before Spark 2.0. It represented the connection to a Spark cluster and was responsible for coordinating the execution of tasks.
# It was used to create RDDs (Resilient Distributed Datasets), which are the fundamental data structures in Spark for distributed computation.
# SparkContext handled low-level functionalities like distributing and scheduling tasks, managing memory, and interacting with storage systems.
# However, with the introduction of SparkSession, most of the functionality of SparkContext was encapsulated within the new session abstraction.

# SparkSession is the entry point for programming Spark applications starting from Spark 2.0.
# It encapsulates both the SparkContext and additional context-related configuration.
# It provides a unified interface for working with structured data using DataFrames and Datasets, which offer optimizations and benefits over RDDs.
# SparkSession also manages various context-specific settings and configurations, like SQL configuration, Hive integration, and more.
# It simplifies the process of working with different Spark libraries (SQL, Streaming, MLlib, GraphX) by providing a consistent API.

# initializing a SparkSession taking in consideration all the cores in local system and giving a name to the Application as "SparkSession"
spark = (
    SparkSession.builder.master("local[*]")
    .appName("PySpark SQL Assignment 01")
    .getOrCreate()
)

# Printing out the value of spark variable that stores the spark session.
spark

**Question 01**


In [ ]:
# Loading the dataset
ratingsRDD = spark.sparkContext.textFile(
    "/content/drive/MyDrive/Contents/Pyspark/PySpark Dataframe/ratings.dat"
)

In [ ]:
# Define schema
schema_string = "UserID MovieID Rating Timestamp"
schema = StructType(
    [
        StructField(field_name, StringType(), True)
        for field_name in schema_string.split()
    ]
)

In [ ]:
# Create DataFrame
row_rdd = ratingsRDD.map(lambda x: x.split("::")).map(
    lambda x: (x[0], x[1], x[2], x[3])
)
ratings = spark.createDataFrame(row_rdd, schema)

In [ ]:
# Show the output
ratings.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows



**Question 02**


In [ ]:
# Loading the dataset
usersRDD = spark.sparkContext.textFile(
    "/content/drive/MyDrive/Contents/Pyspark/PySpark Dataframe/users.dat"
)

In [ ]:
# Define schema
schema_string = "UserID Gender Age Occupation Zip-code"
schema = StructType(
    [
        StructField(field_name, StringType(), True)
        for field_name in schema_string.split()
    ]
)

In [ ]:
# Create DataFrame
row_rdd = usersRDD.map(lambda x: x.split("::")).map(
    lambda x: (x[0], x[1], x[2], x[3], x[4])
)
users = spark.createDataFrame(row_rdd, schema)

In [ ]:
# Show the output
users.show()

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
|     6|     F| 50|         9|   55117|
|     7|     M| 35|         1|   06810|
|     8|     M| 25|        12|   11413|
|     9|     M| 25|        17|   61614|
|    10|     F| 35|         1|   95370|
|    11|     F| 25|         1|   04093|
|    12|     M| 25|        12|   32793|
|    13|     M| 45|         1|   93304|
|    14|     M| 35|         0|   60126|
|    15|     M| 25|         7|   22903|
|    16|     F| 35|         0|   20670|
|    17|     M| 50|         1|   95350|
|    18|     F| 18|         3|   95825|
|    19|     M|  1|        10|   48073|
|    20|     M| 25|        14|   55113|
+------+------+---+----------+--------+
only showing top 20 rows



**Question 03**


In [ ]:
# Loading the dataset
movies = spark.read.text(
    "/content/drive/MyDrive/Contents/Pyspark/PySpark Dataframe/movies.dat"
)

NameError: name 'spark' is not defined

In [ ]:
# Split the columns using '::'
split_columns = movies.withColumn("split_data", split(col("value"), "::"))

In [ ]:
# Extract the MovieID, Title, and Genres (Year) columns
extracted_df = split_columns.select(
    col("split_data")[0].alias("MovieID"),
    col("split_data")[1].alias("Title"),
    col("split_data")[2].alias("Genres_Year"),
)

In [ ]:
# Extract Year and replace it in the Title and Genres columns
year_and_genres_df = (
    extracted_df.withColumn("Year", regexp_extract(col("Title"), "\\((\\d{4})\\)", 1))
    .withColumn("Title", regexp_replace(col("Title"), "\\((\\d{4})\\)", ""))
    .withColumn("Genres", regexp_replace(col("Genres_Year"), "\\((\\d{4})\\)", ""))
    .drop("Genres_Year")
)

In [ ]:
# Show the output
year_and_genres_df.show(truncate=False)

NameError: name 'year_and_genres_df' is not defined

In [ ]:
# Stop the SparkSession
# spark.stop()